In [3]:
import requests, re
from lxml import html
import json

In [104]:
class Dou_interaction:
    
    def __init__(self, url='https://jobs.dou.ua/vacancies/?search=data+engineer'):
        #xpath for getting all links to jobs descriptions
        self.xpath_vacancy_link = "//div[@class='vacancy' and @_id]//a[@class='vt' and @href]"
        # xpath for an element which contains token
        self.xpath_script = "//script[contains(text(),'window.CSRF_TOKEN = \"')]"
        #token
        self.csrf_token = None
        #all links to jobs
        self.urls = []
        #for 1-st request, changing User-Agent
        self.headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:85.0) Gecko/20100101 Firefox/85.0'
            }
        #changing headers for post-request
        self.headers_post = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:85.0) Gecko/20100101 Firefox/85.0',
            'X-Requested-With': 'XMLHttpRequest',
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
            'Referer': 'https://jobs.dou.ua/vacancies/?search=data+engineer',
            }
        #directory for saving html pages
        self.dir = "result\\"
        #link to main page
        self.url = url
        #changing a link for a post request
        self.get_url_post(url)
    
    def get_url_post(self, url):
        start = url.index('?')
        search_query = url[start:]
        self.url_post = 'https://jobs.dou.ua/vacancies/xhr-load/'+search_query.replace('+','%20')
        
    def get_vacancies_links(self):
        #making request with headers, getting text
        self.session = requests.Session()
        search_page = self.session.get(url=self.url, headers=self.headers).text
        root = html.fromstring(search_page)
        
        #getting token
        script = root.xpath(self.xpath_script)[0].text
        pattern = r'window.CSRF_TOKEN = "(.*?)"'
        self.csrf_token = re.findall(pattern, script)[0] 
        
        #getting all links to vacancies
        self.get_links(page=search_page)
        while True:
            #getting more links
            json_response = self.get_add_vacancies(count=len(self.urls))
            #adding them to list of links
            self.get_links(page=json_response.get('html'))
            #if link is the last one - break
            if json_response.get('last'):
                break;

    def get_links(self, page):
        root = html.fromstring(page)
        vacancies_links = root.xpath(self.xpath_vacancy_link)
        for link in vacancies_links:
            self.urls.append(link.attrib['href'])

    def get_add_vacancies(self,count:int):
        #параметрі пост запроса, заполнение
        my_obj = {'csrfmiddlewaretoken': self.csrf_token, 'count': count}
        x = self.session.post(url=self.url_post, data=my_obj, headers=self.headers_post)
        return x.json()
    
    def get_page_content(self, url:str):
        page = self.session.get(url, headers=self.headers).text
        return page
    
    def get_and_save_all_pages(self):
        self.get_vacancies_links()
        
        for num, url in enumerate(self.urls):
            text_page = self.get_page_content(url=url)
            with open(self.dir+"page_"+str(num)+".html",'wt', encoding='utf8') as file:
                file.write(text_page)

In [105]:
dou_int = Dou_interaction()
dou_int.get_and_save_all_pages()